In [3]:
!pip install pyspark

In [5]:
!pip install tweepy

  Using cached tweepy-4.12.1-py3-none-any.whl (101 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.25.0
    Uninstalling requests-2.25.0:
      Successfully uninstalled requests-2.25.0


You should consider upgrading via the 'c:\users\furka\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install findspark

  Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)


You should consider upgrading via the 'c:\users\furka\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [1]:
# May cause deprecation warnings, safe to ignore, they aren't errors
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)
socket_stream = ssc.socketTextStream("127.0.0.1", 5554)
lines = socket_stream.window( 20 )
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) ) # Registers to a table.

C:\Spark\python\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
ssc.start()    